# Lab 11

## Connect to DBMS

In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost")
collection = client['test']['restaurants']

## Exercise 1

### Query all Irish cuisines

In [2]:
collection.find({'cuisine': 'Irish'})

To show data we can use `list()` or `for`. Then, I'm going to use `list()` with limit.

In [3]:
list(collection.find({'cuisine': 'Irish'}))[:5]

[{'_id': ObjectId('642c0efcc8d9f8bb05ccd5f0'),
  'address': {'building': '351',
   'coord': [-73.98513559999999, 40.7676919],
   'street': 'West   57 Street',
   'zipcode': '10019'},
  'borough': 'Manhattan',
  'cuisine': 'Irish',
  'grades': [{'date': datetime.datetime(2014, 9, 6, 0, 0),
    'grade': 'A',
    'score': 2},
   {'date': datetime.datetime(2013, 7, 22, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2012, 7, 31, 0, 0), 'grade': 'A', 'score': 12},
   {'date': datetime.datetime(2011, 12, 29, 0, 0), 'grade': 'A', 'score': 12}],
  'name': 'Dj Reynolds Pub And Restaurant',
  'restaurant_id': '30191841'},
 {'_id': ObjectId('642c0efcc8d9f8bb05ccd648'),
  'address': {'building': '15',
   'coord': [-73.9896713, 40.7287978],
   'street': 'East    7 Street',
   'zipcode': '10003'},
  'borough': 'Manhattan',
  'cuisine': 'Irish',
  'grades': [{'date': datetime.datetime(2014, 6, 7, 0, 0),
    'grade': 'A',
    'score': 8},
   {'date': datetime.datetime(2014, 1, 9, 0, 0

### Query all Irish and Russian cuisines

In [4]:
list(collection.find({'cuisine': {'$in': ['Irish', 'Russian']}}))[:5]

[{'_id': ObjectId('642c0efcc8d9f8bb05ccd5f0'),
  'address': {'building': '351',
   'coord': [-73.98513559999999, 40.7676919],
   'street': 'West   57 Street',
   'zipcode': '10019'},
  'borough': 'Manhattan',
  'cuisine': 'Irish',
  'grades': [{'date': datetime.datetime(2014, 9, 6, 0, 0),
    'grade': 'A',
    'score': 2},
   {'date': datetime.datetime(2013, 7, 22, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2012, 7, 31, 0, 0), 'grade': 'A', 'score': 12},
   {'date': datetime.datetime(2011, 12, 29, 0, 0), 'grade': 'A', 'score': 12}],
  'name': 'Dj Reynolds Pub And Restaurant',
  'restaurant_id': '30191841'},
 {'_id': ObjectId('642c0efcc8d9f8bb05ccd648'),
  'address': {'building': '15',
   'coord': [-73.9896713, 40.7287978],
   'street': 'East    7 Street',
   'zipcode': '10003'},
  'borough': 'Manhattan',
  'cuisine': 'Irish',
  'grades': [{'date': datetime.datetime(2014, 6, 7, 0, 0),
    'grade': 'A',
    'score': 8},
   {'date': datetime.datetime(2014, 1, 9, 0, 0

### Find a restaurant with the following address: Prospect Park West 284, 11215

In [5]:
collection.find_one({'address.street': 'Prospect Park West', 'address.building': '284', 'address.zipcode': '11215'})

{'_id': ObjectId('642c0efcc8d9f8bb05ccd601'),
 'address': {'building': '284',
  'coord': [-73.9829239, 40.6580753],
  'street': 'Prospect Park West',
  'zipcode': '11215'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 11, 19, 0, 0),
   'grade': 'A',
   'score': 11},
  {'date': datetime.datetime(2013, 11, 14, 0, 0), 'grade': 'A', 'score': 2},
  {'date': datetime.datetime(2012, 12, 5, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2012, 5, 17, 0, 0), 'grade': 'A', 'score': 11}],
 'name': 'The Movable Feast',
 'restaurant_id': '40361606'}

## Exercise 2
Insert into the database following restaurant:
- Address: Sportivnaya 126, 420500, -73.9557413, 40.7720266
- Borough: Innopolis
- Cuisine: Serbian
- Name: The Best Restaurant
- Id: 41712354
- Grades:
    - A, 11, 04 Apr, 2023

In [6]:
import datetime
data = {
    'address': {'building': '126', 'coord': [-73.9557413, 40.7720266], 'street': 'Sportivnaya', 'zipcode': '420500'},
    'borough': 'Innopolis', 'cuisine': 'Serbian',
    'grades': [{'date': datetime.datetime(2023, 4, 4, 0, 0), 'grade': 'A', 'score': 11}],
    'name': 'The Best Restaurant', 'restaurant_id': '41712354'}
result = collection.insert_one(data)

In [7]:
result.inserted_id

ObjectId('642c15625e7b10704f606a93')

## Exercise 3

### Delete from the database a single Brooklyn located restaurant

In [8]:
collection.delete_one({'borough': 'Brooklyn'}).raw_result

{'n': 1, 'ok': 1.0}

### Delete from the database all Thai cuisines

In [9]:
collection.delete_many({'cuisine': 'Thai'}).raw_result

{'n': 0, 'ok': 1.0}

## Exercise 4
Query all restaurants on the  Prospect Park West street, for each of them do the following:
- if it has more than one A grades → delete this restaurant
- otherwise, add another A grade to this restaurant

In [10]:
rests_on_street = collection.find({'address.street': 'Prospect Park West', 'grades.grade': 'A'})

In [11]:
rests_with_aa = []
for e in rests_on_street:
    a_counter = 0
    for g in e['grades']:
        if g['grade'] == 'A':
            a_counter += 1
    if a_counter > 1:
        collection.delete_one(e)
    else:
        collection.update_one(e, {'$push': {'grades': {'date': datetime.datetime(2023, 4, 4, 0, 0), 'grade': 'A', 'score': 11}}})